# **SQLAlchemy**

SQLAlchemy is an open-source Python library that provides a set of tools for working with SQL databases.\
It allows developers to interact with databases using Python code rather than writing SQL statements directly.

**Core** - foundation, general toolkit\
**ORM** - Object Relational Mapping

ORM allows Python to map classes to database tables and rows, making it easy to perform CRUD (Create, Read, Update, Delete) operations. ORM also provides features like relationships, lazy loading, and transactions.

---

**Engine object** - Establish connection, created once:\
https://docs.sqlalchemy.org/en/20/tutorial/engine.html

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///my-database.db') # <- Connection string

DBAPI - "Driver" for SQL interaction

Example: <a href="https://learn.microsoft.com/en-us/sql/odbc/microsoft-open-database-connectivity-odbc?view=sql-server-ver16">(Microsoft Open Database Connectivity (ODBC)</a>

### Connection string

A connection string typically includes information such as the type of driver, the server name, the database name, the user name, the password, and other options. A connection string can be specified in different formats depending on the data provider or driver.

https://www.connectionstrings.com



### Working with transactions

By using with <code>enginge.connect()</code> as connection: 

In [3]:
from sqlalchemy import text

with engine.connect() as c:
    result = c.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


### Committing Changes
DBAPI connections are non-autocommitting. Need to use <code>commit()</code>.

In [4]:
with engine.connect() as c:
    c.execute(text("CREATE TABLE some_table (x int, y int)"))
    c.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    c.commit()

### Fetching Rows

In [5]:
with engine.connect() as c:
    result = c.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

x: 1  y: 1
x: 2  y: 4


### Sending parameters

Always send data and code separated.

In [6]:
with engine.connect() as c:
    result = c.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

x: 2  y: 4


---
## **⚠️ Injections & Parameter Bindings**

### Parameter binding - used to safely insert user input and avoid

In [ ]:
with engine.connect() as c:
    result = c.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": userY}) # <-- Parameter binding, GOOD
    result = c.execute(text("SELECT x, y FROM some_table WHERE y > 2") + userY) # <-- Concat, BAD

    for row in result:
        print(f"x: {row.x} y:{row.y}")

In the second c.execute line, the concatenation of the userY variable to the end of the query string is unsafe and can lead to SQL injection attacks. Instead, we should use *parameter binding*, as in the first c.execute line.

With parameter binding we use placeholders in the query string, and then provide the actual values for those placeholders separately. The placeholders are typically represented by a colon followed by a parameter name.


#### In T-SQL we would use @Y instead of :y


>**Always use bound parameters**\
As mentioned at the beginning of this section, textual SQL is not the usual way we work with SQLAlchemy. However, when using textual SQL, a Python literal value, even non-strings like integers or dates, should never be stringified into SQL string directly; a parameter should always be used. This is most famously known as how to avoid SQL injection attacks when the data is untrusted. However it also allows the SQLAlchemy dialects and/or DBAPI to correctly handle the incoming input for the backend. Outside of plain textual SQL use cases, SQLAlchemy’s Core Expression API otherwise ensures that Python literal values are passed as bound parameters where appropriate.*

*https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html#sending-parameters

Always set up restrictions and also separate code from data.

---

## Database metadata

The central element of both SQLAlchemy Core and ORM is the SQL Expression Language which allows for fluent, composable construction of SQL queries. The foundation for these queries are Python objects that represent database concepts like tables and columns. These objects are known collectively as database metadata.

We *declare* or use *reflection*.

- Reflection automatically generate the metadata object by reading the db.
- Declaration is used 'manually'.

Having a single MetaData object for an entire application is the most common case, represented as a module-level variable in a single place in an application, often in a “models” or “dbschema” type of package.

In [15]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [16]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [17]:
user_table.c.keys() # Get keys

['id', 'name', 'fullname']

### Declaring constraints

In [18]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [19]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

### Emitting DDL to the Database

When we want to send Data Definition Language to the database.

The first useful thing we can do with this structure will be to emit CREATE TABLE statements, or DDL, to our SQLite database so that we can insert and query data from them. We have already all the tools needed to do so, by invoking the MetaData.<code>create_all()</code> method on our MetaData, sending it the Engine that refers to the target database:

In [20]:
metadata_obj.create_all(engine)